In [1]:
# Sentence encodings
from bert_sent_encoding import bert_sent_encoding


# Import pre-trained Bert for sentence embeds
bse = bert_sent_encoding(model_path='bert-base-cased')

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [109]:
import logging
import os
import queue
import random
import re
from args import get_setup_args
import shutil
import string
#import setup
import torch
import torch.nn.functional as F
import torch.utils.data as data
from collections import Counter
import tqdm
import numpy as np
import ujson as json
#import spacy
import json




In [110]:
def pair_question_contexts():
# Read in train test, dev jsons
# Get each paragraph

# TRAIN
# Split into {question_id -> ("question_string", ["sent1_str", "sent2_str", "sent3_str", ...]))}
# Split into {qid -> (q_embed,[sent1_embed, sent2_embed, sent3_embed, ....])}

#DEV + TEST
# Do same for dev + test

#populate three dictionaries

    datasets = ["dev","test","train"]
    dev_data = train_data = test_data = None
    v_str = "-v2.0.json"

    for ds in datasets:
        filename = ds + v_str
        with open(filename) as f:
            json_file = json.load(f)

                

            # outermost is a keys list of [version, data]
            data = json_file["data"]
            # Data is a list of dicts
            for entry in data:
            
                # Each entry is a dict of keys [title, paragraphs]
#                 print(entry['title']) #Title example is normans
                #Paragraphs is a list of dicts
    #            print(ex_entry['paragraphs'])


                paragraphs = entry['paragraphs']
    #             print(ex_paragraphs[0])
            
                # Want to create new paragraphs list 
                context_to_qa_list = [] # REPLACE paragraphs
                # Each paragraph has a list of questions and associated answers and context
                # each paragraph is a dict with keys [qas, context]  
                #ex_paragraph['context'] is just sent paragraph
    #             print(ex_paragraph['qas'])
                for paragraph in paragraphs:
                    context = paragraph['context']   
                    for qa in paragraph['qas']:
                        # Append to 1:1 dict list here
                        context_to_qa_list.append({'qas':qa,'context':context})
                    
                entry['paragraphs'] = context_to_qa_list
                
                if ds == "dev":
                    dev_data = json_file
#                     print(dev_data['data'][0]['paragraphs'][0])
                elif ds == "test":
                    test_data = json_file
                else:
                    train_data = json_file
            
    return dev_data,train_data,test_data

#             vectors = bse.get_vector(['sent2', 'sent3'], word_vector=False, layer=-1)
#             print(vectors)
#             vectors = bse.get_vector(['sent2', 'sent3'], word_vector=False, layer=-1)  # 4th line 2. get vector list of strings
#            bse.write_txt2vector(input_file, output_file, word_vector=False, layer=-1)   # 5th line 3. get and write vectors of strings


In [111]:
# Maps all dev,train, and test to 1:1 in regards to question and context
# no longer have multiple questions per context, now ea. q-a gets its own context :3
data = pair_question_contexts()


In [112]:
import re
from scipy.spatial.distance import cosine as cos_sim
from tqdm import tqdm_notebook

In [113]:
# FOR NOW COSINE SIM, but NEXT STEP IS REPLACING COS_SIM WITH MODEL PREDICTION
def remove_least_relevant_sents(data):
    for dataset in data:
        for entry in dataset['data']:
            paragraphs = entry['paragraphs']
            context_to_qa_list = []
            for paragraph in paragraphs:
                punct = '!.?'
                curr_context = paragraph['context']
                curr_qa = paragraph['qas']
                
                question = curr_qa['question']
                q_embed = bse.get_vector(question, word_vector=False, layer=-1)
                
                # THIS IS IN CASE OF PUNCTUATION INSIDE OF QUOTES(for dialogue) NOT FINISHED 
    #             context_as_sentences = None
    #             if '"' in curr_context or "'" in curr_context:
    #                 lst = []
    #                 prev_index = 0
    #                 has_closed = True
    #                 for i in range(curr_context):
    #                     if curr_context[i] == """""" or curr_context[i] == "''":
    #                         has_closed = not has_closed
    #                     if curr_context[i] in punct:
    #                         context_as_sentences.append(context[prev_index: i])
    #                 context_as_sentences = lst
                
                # Split up context into sentences
                context_as_sentences = re.split(r"([!|.|?])+",curr_context)
                context_as_sentences, punctuation = context_as_sentences[0::2], context_as_sentences[1::2]
                

                # Remove empty elems in lst
                context_as_sentences = list(filter(lambda x: x != "",context_as_sentences))

                # Get context embeddings
                context_embeds = bse.get_vector(context_as_sentences, word_vector=False, layer=-1)
                
                #Note that spatial.distance.cosine computes the distance, and not the similarity. 
                #So, you must subtract the value from 1 to get the similarity

                lowest_sent_sim = 1.1 
                sent_index_to_remove = -1
                preds = []
                # Find worst sentence
                for sent_embed in context_embeds:
                    embed_similarity = 1 - cos_sim(q_embed, sent_embed)
                    preds.append(embed_similarity)
#                     if embed_similarity < lowest_sent_sim:
#                         sent_index_to_remove = index
                sent_index_to_remove = np.argmax(np.asarray(preds))
                        
                print(preds)
                # Remove least relevant sentence
                print("CONTEXT IS = " + curr_context)
                print(sent_index_to_remove)
                print("QUESTION is = " + str(curr_qa))
                print("sent removed is = " + str(context_as_sentences[sent_index_to_remove]))
                context_as_sentences.pop(sent_index_to_remove)
                punctuation.pop(sent_index_to_remove)
                
                new_context = ''.join([sent + punc for (sent,punc) in zip(context_as_sentences,punctuation)])
                context_to_qa_list.append({'qas':curr_qa,'context':new_context})
                entry['paragraphs'] = context_to_qa_list
                
    return data
    
            

In [114]:
worst_relevant_sent_removed = remove_least_relevant_sents(data)
print(worst_relevant_sent_removed[0]['data'][0]['paragraphs'])

[0.9582187345613867, 0.7938299883305213, 0.7528911884357551, 0.9815255150529854]
CONTEXT IS = The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
3
QUESTION is = {'question': 'In what country is Normandy located?', 'id': '68cf05f67fd29c6f129fe2fb9', 'answers': [{'text': 'France', 'answer_start': 159}, {'text

[0.9739697908273132, 0.8469607363571628, 0.8086626879318521, 0.9651287109598927]
CONTEXT IS = The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
0
QUESTION is = {'plausible_answers': [{'text': 'Rollo', 'answer_start': 308}], 'question': 'Who did King Charles III swear fealty to?', 'id': 'e06fbbdb50af7ab3fae

[0.9814161516175026, 0.9859437338777989, 0.9639682285716835, 0.7495480839354742, 0.9710983876866245, 0.8260775225598037, 0.6701357863411738]
CONTEXT IS = The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy into which they assimilated. They adopted the Gallo-Romance language of the Frankish land they settled, their dialect becoming known as Norman, Normaund or Norman French, an important literary language. The Duchy of Normandy, which they formed by treaty with the French crown, was a great fief of medieval France, and under Richard I of Normandy was forged into a cohesive and formidable principality in feudal tenure. The Normans are noted both for their culture, such as their unique Romanesque architecture and musical traditions, and for their significant military accomplishments and innovati

[0.019769758864039755, -0.019769758864039755, -0.019769758864039755, 0.019769758864039644, -0.019769758864039755, -0.019769758864039755, -0.019769758864039755, 0.019769758864039755, 0.019769758864039755, 0.019769758864039755, -0.019769758864039755, -0.019769758864039755, 0.019769758864039755, 0.019769758864039755, -0.019769758864039755, 0.019769758864039755, -0.019769758864039755, 0.019769758864039644, 0.019769758864039755, 0.019769758864039755, 0.019769758864039644, 0.019769758864039755, 0.019769758864039755, -0.019769758864039755, -0.019769758864039755, 0.019769758864039755, -0.019769758864039755, -0.019769758864039755, -0.019769758864039755, 0.019769758864039755, 0.019769758864039755, 0.019769758864039755, -0.019769758864039755, 0.019769758864039755, 0.019769758864039755, -0.019769758864039755, -0.019769758864039755, -0.019769758864039755, -0.019769758864039755, 0.019769758864039755, 0.019769758864039755, 0.019769758864039755, -0.019769758864039755, 0.019769758864039755, -0.01976975

[0.02129431888922728, -0.02129431888922717, -0.02129431888922717, 0.02129431888922728, -0.02129431888922717, -0.02129431888922717, -0.02129431888922717, 0.02129431888922728, 0.02129431888922728, 0.02129431888922728, -0.02129431888922717, -0.02129431888922717, 0.02129431888922728, 0.02129431888922728, -0.02129431888922717, 0.02129431888922728, -0.02129431888922717, 0.02129431888922728, 0.02129431888922728, 0.02129431888922728, 0.02129431888922728, 0.02129431888922728, 0.02129431888922728, -0.02129431888922717, -0.02129431888922717, 0.02129431888922728, -0.02129431888922717, -0.02129431888922717, -0.02129431888922717, 0.02129431888922728, 0.02129431888922728, 0.02129431888922728, -0.02129431888922717, 0.02129431888922728, 0.02129431888922728, -0.02129431888922717, -0.02129431888922717, -0.02129431888922717, -0.02129431888922717, 0.02129431888922728, 0.02129431888922728, 0.02129431888922728, -0.02129431888922717, 0.02129431888922728, -0.02129431888922717, 0.02129431888922728, 0.0212943188

...

[0.9318244483600425, 0.9896228819465496, 0.9647871898738263, 0.9791701080768948, 0.9926551668872944, 0.9633022037090714, 0.9538185877254233, 0.9720079981523992, 0.9806713262222366, 0.9878711985694569, 0.991444208876215, 0.956750307299153]
CONTEXT IS = The Normans had a profound effect on Irish culture and history after their invasion at Bannow Bay in 1169. Initially the Normans maintained a distinct culture and ethnicity. Yet, with time, they came to be subsumed into Irish culture to the point that it has been said that they became "more Irish than the Irish themselves." The Normans settled mostly in an area in the east of Ireland, later known as the Pale, and also built many fine castles and settlements, including Trim Castle and Dublin Castle. Both cultures intermixed, borrowing from each other's language, culture and outlook. Norman descendants today can be recognised by their surnames. Names such as French, (De) Roche, Devereux, D'Arcy, Treacy and Lacy are particularly common in th

[0.876589674785226, 0.6115451495542933, 0.9856484633835846, 0.7912429316556691]
CONTEXT IS = One of the claimants of the English throne opposing William the Conqueror, Edgar Atheling, eventually fled to Scotland. King Malcolm III of Scotland married Edgar's sister Margaret, and came into opposition to William who had already disputed Scotland's southern borders. William invaded Scotland in 1072, riding as far as Abernethy where he met up with his fleet of ships. Malcolm submitted, paid homage to William and surrendered his son Duncan as a hostage, beginning a series of arguments as to whether the Scottish Crown owed allegiance to the King of England.
2
QUESTION is = {'plausible_answers': [{'text': 'William', 'answer_start': 272}], 'question': 'Who invaded Scotland in the 10th century?', 'id': '312db27fb16fe0afa64cfb718', 'answers': [], 'is_impossible': True}
sent removed is =  William invaded Scotland in 1072, riding as far as Abernethy where he met up with his fleet of ships
[0.932970

[0.9720858647834658, 0.9179425491081515, 0.9759448988283194]
CONTEXT IS = Even before the Norman Conquest of England, the Normans had come into contact with Wales. Edward the Confessor had set up the aforementioned Ralph as earl of Hereford and charged him with defending the Marches and warring with the Welsh. In these original ventures, the Normans failed to make any headway into Wales.
2
QUESTION is = {'question': 'Where was Ralph earl of?', 'id': 'ed3c89d847cae4d25c5f4fed9', 'answers': [{'text': 'Hereford', 'answer_start': 158}, {'text': 'Hereford', 'answer_start': 158}, {'text': 'Hereford', 'answer_start': 158}], 'is_impossible': False}
sent removed is =  In these original ventures, the Normans failed to make any headway into Wales
[0.9688328789646968, 0.925004205832668, 0.9741504923813247]
CONTEXT IS = Even before the Norman Conquest of England, the Normans had come into contact with Wales. Edward the Confessor had set up the aforementioned Ralph as earl of Hereford and charged hi

[0.9026578756076992, 0.9903390536678143, 0.9868977010587857, 0.8627696240774513]
CONTEXT IS = The legendary religious zeal of the Normans was exercised in religious wars long before the First Crusade carved out a Norman principality in Antioch. They were major foreign participants in the Reconquista in Iberia. In 1018, Roger de Tosny travelled to the Iberian Peninsula to carve out a state for himself from Moorish lands, but failed. In 1064, during the War of Barbastro, William of Montreuil led the papal army and took a huge booty.
1
QUESTION is = {'plausible_answers': [{'text': 'Antioch', 'answer_start': 142}], 'question': 'Where did the Normans carve out a principality before the First Crusade?', 'id': '9f0cc4c0dad2bd7a4bcecbdde', 'answers': [], 'is_impossible': True}
sent removed is =  They were major foreign participants in the Reconquista in Iberia
[0.8587602230294662, 0.9700932133166911, 0.9714353484293106, 0.814490996483486]
CONTEXT IS = The legendary religious zeal of the Norman

IndexError: pop index out of range